In [1]:
%cd 'Q:\Dropbox\python\QSim\'

Q:\Dropbox\python\QSim


In [4]:
%matplotlib inline
from scipy.io import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from scipy.integrate import ode

In [5]:
def cross(a, b):
    return [a[i]*b[i+1]-a[i+1]*b[i] for i in [1,-1,0]]

def jacobian(X):
    J = [[(np.roll(X[i],-1,j)-np.roll(X[i],1,j))*0.5 for j in range(0,3)] for i in range(0,3)]
    for i in range(0,3):
        J[i][0][0,:,:] = -1.5*X[i][0,:,:]+2*X[i][1,:,:]-0.5*X[i][2,:,:]
        J[i][0][-1,:,:] = 1.5*X[i][-1,:,:]-2*X[i][-2,:,:]+0.5*X[i][-3,:,:]
        J[i][1][:,0,:] = -1.5*X[i][:,0,:]+2*X[i][:,1,:]-0.5*X[i][:,2,:]
        J[i][1][:,-1,:] = 1.5*X[i][:,-1,:]-2*X[i][:,-2,:]+0.5*X[i][:,-3,:]
        J[i][2][:,:,0] = -1.5*X[i][:,:,0]+2*X[i][:,:,1]-0.5*X[i][:,:,2]
        J[i][2][:,:,-1] = 1.5*X[i][:,:,-1]-2*X[i][:,:,-2]+0.5*X[i][:,:,-3]
    return J

def hessian(X):
    J = jacobian(X)   
    H = [[[(-2*X[i] + np.roll(X[i],-1,k)+np.roll(X[i],1,k)) if k == j else 
           (np.roll(J[i][j],-1,k)-np.roll(J[i][j],1,k))*0.5 for k in range(0,j+1)] for j in range(0,3)] for i in range(0,3)]
    
    for i in range(0,3):
        H[i][0][0][0,:,:] = 2*X[i][0,:,:]-5*X[i][1,:,:]+4*X[i][2,:,:]-X[i][3,:,:]
        H[i][0][0][-1,:,:] = 2*X[i][-1,:,:]-5*X[i][-2,:,:]+4*X[i][-3,:,:]-X[i][-4,:,:]
        H[i][1][1][:,0,:] = 2*X[i][:,0,:]-5*X[i][:,1,:]+4*X[i][:,2,:]-X[i][:,3,:]
        H[i][1][1][:,-1,:] = 2*X[i][:,-1,:]-5*X[i][:,-2,:]+4*X[i][:,-3,:]-X[i][:,-4,:]
        H[i][2][2][:,:,0] = 2*X[i][:,:,0]-5*X[i][:,:,1]+4*X[i][:,:,2]-X[i][:,:,3]
        H[i][2][2][:,:,-1] = 2*X[i][:,:,-1]-5*X[i][:,:,-2]+4*X[i][:,:,-3]-X[i][:,:,-4]
        
        for j in (1,2):
            H[i][j][0][0,:,:] = -1.5*J[i][j][0,:,:]+2*J[i][j][1,:,:]-0.5*J[i][j][2,:,:]
            H[i][j][0][-1,:,:] = 1.5*J[i][j][-1,:,:]-2*J[i][j][-2,:,:]+0.5*J[i][j][-3,:,:]
        
        H[i][2][1][:,0,:] = -1.5*J[i][2][:,0,:]+2*J[i][2][:,1,:]-0.5*J[i][2][:,2,:]
        H[i][2][1][:,-1,:] = 1.5*J[i][2][:,-1,:]-2*J[i][2][:,-2,:]+0.5*J[i][2][:,-3,:]        
    
    return (H, J)       

def det(a):
    return np.sum([a[0][i-1]*(a[1][i]*a[2][i+1]-a[1][i+1]*a[2][i]) for i in [1,-1,0]],0)

def rot(B, X):
    hes = hessian(X)
    J = hes[1]
    H = hes[0]
    detJ = det(J)
    Jb = [[(J[i][j]*J[i+1][j+1] - J[i+1][j]*J[i][j+1])/detJ for j in [-2,-1,0]] for i in [1,-1,0]]
    dB = [np.gradient(B[i]) for i in range(0,3)]
    Bx = [np.sum([J[i][j]*B[j] for j in range(0,3)]/detJ,0) for i in range(0,3)]
    
    #B2 = np.sum(np.power(B,2),0)
    #Bx2 = np.sum(np.power(Bx,2),0)
  
    DdetJ = [det([[H[i][j][0],J[i][1],J[i][2]] for i in range(0,3)])+
            det([[J[i][0],H[i][max(j,1)][min(j,1)],J[i][2]] for i in range(0,3)])+
            det([[J[i][0],J[i][1],H[i][max(j,2)][min(j,2)]] for i in range(0,3)]) for j in range(0,3)]

    dBx = [[(np.sum([J[i][k]*dB[k][j] + H[i][max(k,j)][min(k,j)]*B[k] for k in range(0,3)],0) -
                DdetJ[j]*Bx[i])/detJ for j in range(0,3)] for i in range(0,3)]
    
    rotB = [np.sum([Jb[i][j]*dBx[i+1][j]-Jb[i+1][j]*dBx[i][j] for j in range(0,3)],0) for i in [1,-1,0]]
    
    force = [np.sum([B[j]*dBx[i][j]/detJ - np.sum([Bx[k]*dBx[k][j] for k in range(0,3)],0)*Jb[i][j]
                     for j in range(0,3)],0) for i in range(0,3)]
    
    #b = [np.sum([B[j]*dBx[i][j]/detJ for j in range(0,3)],0) for i in range(0,3)]
    #c = [np.sum([Bx[j]*dBx[j][i] for j in range(0,3)],0)  for i in range(0,3)]
    
    
    #q = [np.sum([(B2/detJ**2*(i == j) - Bx[i]*np.sum([Jb[k][j]*B[k] for k in range(0,3)],0)/detJ - 
    #      Bx[j]*np.sum([Jb[k][i]*B[k] for k in range(0,3)],0)/detJ + 
    #      Bx2*np.sum([Jb[k][i]*Jb[k][j] for k in range(0,3)],0))*X[j]
    #      for j in range(0,3)],0) for i in range(0,3)]
    
    return force

def force(B, X):
    ## from Craig & Sneyd (1986)
    dB = [np.gradient(B[i]) for i in range(0,3)]
    hes = hessian(X)
    J = hes[1]
    H = hes[0]
    detJ = det(J)
    detJ2 = detJ**2
    Jb = [[(J[i][j]*J[i+1][j+1] - J[i+1][j]*J[i][j+1])/detJ for j in [-2,-1,0]] for i in [1,-1,0]]
    b = [np.sum([J[i][j]*B[j] for j in range(0,3)],0) for i in range(0,3)]
    b2 = np.sum(np.power(b,2),0)
    g = [[1 if i==j else 0 for j in range(0,3)] for i in range(0,3)]
    
    A = [[[[(g[i][j]*B[k]*B[l] - (b[i]*Jb[j][k]+b[j]*Jb[i][k])*B[l] + b2*Jb[l][i]*Jb[j][k])/detJ2
            for l in range(0,3)] for k in range(0,3)] for j in range(0,3)] for i in range(0,3)]
    
    C = [(np.sum([np.sum([B[k]*dB[j][k] for k in range(0,3)],0)*J[i][j] for j in range(0,3)],0) -
         np.sum([np.sum([np.sum([J[k][l]*dB[l][j] for l in range(0,3)],0)*b[k] for k in range(0,3)],0)*Jb[i][j]
                 for j in range(0,3)],0))/detJ2
         for i in range(0,3)]
    
    
    return [np.sum([np.sum([np.sum([A[i][j][k][l]*H[j][max(k,l)][min(k,l)] for l in range(0,3)],0)
                            for k in range(0,3)],0) for j in range(0,3)],0) + C[i] for i in range(0,3)]


In [10]:
dim = [64,64,64]
X = np.mgrid[0:dim[0], 0:dim[1], 0:dim[2]].astype(np.float64)
xsh = X.shape
B = [np.zeros(dim) for i in range(0,3)]
B[2] = np.ones(dim)
X1 = np.array(X)
ar = 8**2
az = 8**2
h = np.exp(-((X[0]-dim[0]/2)**2 + (X[1]-dim[1]/2)**2)/ar - (X[2])**2/az)*np.pi/2

#h = np.exp(-((X[0]-dim[0]/2)**2 + (X[1]-dim[1]/2)**2)/ar)*X[2]/dim[2]*np.pi/4

X[0] = (X1[0]-dim[0]/2)*np.cos(h) - (X1[1]-dim[1]/2)*np.sin(h) + dim[0]/2
X[1] = (X1[0]-dim[0]/2)*np.sin(h) + (X1[1]-dim[1]/2)*np.cos(h) + dim[1]/2

#X[1] = X[1] + (X[0]-dim[0]/2)*np.exp(-np.abs(X[0]-dim[0]/2)/2)*np.cos(np.pi*X[2]/dim[2])

X1 = np.array(X)




In [12]:
def f(t,y):
    y.shape = xsh
    return np.array(rot(B,y)).flatten()

r = ode(f).set_integrator('dop853', verbosity = 1).set_initial_value(X.flatten(),0)
Y = r.integrate(500)
Y.shape = xsh

r.successful()

C:\Anaconda\lib\site-packages\scipy\integrate\_ode.py:949: UserWarning: dop853: step size becomes too small
  self.messages.get(idid, 'Unexpected idid=%s' % idid))


False

In [ ]:
for i in range(0,0):
    F1 = np.array(rot(B,X))
    F2 = np.array(rot(B,X+F1*step*0.5))
    F3 = np.array(rot(B,X+F2*step*0.5))
    F4 = np.array(rot(B,X+F3*step))
    
    F = (F1+2*F2+2*F3+F4)/6
    F[:,:,:,0] = 0
    F[:,:,:,dim[2]-1] = 0
    X += F*step
    print np.max(np.abs(F))

In [17]:
%matplotlib qt


fig = plt.figure(1,[8,8])
ax = fig.add_subplot(111, projection='3d')
for i in range(24,40,2):
    for j in range(24,40,2):
        ax.plot(Y[0][i,j,:],Y[1][i,j,:],Y[2][i,j,:], color = 'green')
plt.show()